In [ ]:
from unidecode import unidecode

import numpy as np
import pandas as pd

# 1. Importação e junção dos datasets

## 1.1. Dados de Internet

### SIMET_VELOC
- **cod_setor:** setor censitário da região avaliada.
- **lat, lon:** coordenadas geográficas.
- **asn:** código da operadora.
- **asn_name:** nome da operadora.
- **type:** tipo de conexão -- *broadband* para banda-larga fixa, *mobile* para banda-larga móvel.
- **numDisps:** número de dispositivos que compõem a amostra.
- **tcp_range_(velocidade):** número de dispositivos por faixa de velocidade de download, em Mb/s.

### DIC_IBGE
- **cod_setor:** setor censitário da região.
- **cod_ap:** área de ponderação da região.
- **cod_mun:** código do município.
- **nom_mun:** nome completo do município.
- **sig_uf:** sigla da unidade federativa.

Os códigos de geolocalização `cod_mun`, `cod_ap` e `cod_setor` são compilados pelo IBGE e disponibilizados na página sobre [Malha de Setores Censitários](https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/26565-malhas-de-setores-censitarios-divisoes-intramunicipais.html?=&t=o-que-e).

### 1.1.1 Importação e geolocalização

In [ ]:
internet_connection = pd.read_parquet('datasets/raw/internet_connection.parquet')
display(internet_connection.head())

dicionario_ibge = pd.read_parquet('datasets/raw/dicionario_ibge.parquet')
display(dicionario_ibge.head())

In [ ]:
dicionario_ibge['nom_mun'] = dicionario_ibge['nom_mun'].apply(lambda x: unidecode(x))
internet_connection = internet_connection.merge(dicionario_ibge, how='inner', on='cod_setor')
internet_connection.head()

### 1.1.2. Agregação por município
Aqui, agrupamos os registros por município e tipo de conexão, e somamos o total de dispositivos por faixa de velocidade (`tcp_range_...`).

In [ ]:
tcp_range_cols = [x for x in internet_connection.columns.tolist() if 'tcp_range' in x]
agg_dict = {key: 'sum' for key in tcp_range_cols}
agg_dict['asn_name'] = 'nunique'

internet_connection_grp = internet_connection.groupby(['cod_mun', 'type', 'nom_mun', 'sig_uf']).agg(agg_dict).reset_index().rename(columns={'asn_name': 'num_operadoras'})
internet_connection_grp.head()

## 1.2. Dados de Educação

### DIC_IBGE
- **cod_setor:** setor censitário da região.
- **cod_ap:** área de ponderação da região.
- **cod_mun:** código do município.
- **nom_mun:** nome completo do município.
- **sig_uf:** sigla da unidade federativa.

Os códigos de geolocalização `cod_mun`, `cod_ap` e `cod_setor` são compilados pelo IBGE e disponibilizados na página sobre [Malha de Setores Censitários](https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/26565-malhas-de-setores-censitarios-divisoes-intramunicipais.html?=&t=o-que-e).
Este dataframe é o mesmo utilizado na seção anterior.

In [ ]:
def remove_multilevel_index(df):
    for i in range(len(df.columns.values)):
        col = list(df.columns.values[i])

        # Transforma títulos em string
        for j in range(3):
            col[j] = str(col[j])

        # Retira títulos vazios
        for j in range(3):
            if 'Unnamed' in col[j]:
                col[j] = ""

        col = tuple(col)
        df.columns.values[i] = col

    # Junta os títulos hierárquicos em um só
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    
    return df

### 1.2.1. IDEB para anos iniciais

In [ ]:
dados_ideb_ef_1 = pd.read_excel('datasets/raw/divulgacao_anos_iniciais_municipios_2019.xlsx',
                                header = [6, 7, 8])

dados_ideb_ef_1 = remove_multilevel_index(dados_ideb_ef_1)
dados_ideb_ef_1 = dados_ideb_ef_1[dados_ideb_ef_1['Rede'] == 'Pública'][[
    'Código do Município',
    'Taxa de Aprovação - 2019  5º',
    'Nota SAEB - 2019 Matemática',
    'Nota SAEB - 2019 Língua Portuguesa',
    'IDEB\n2019\n(N x P)'
]].rename(columns={
    'Código do Município': 'cod_mun',
    'Taxa de Aprovação - 2019  5º': 'tx_aprov_ef_1',
    'Nota SAEB - 2019 Matemática': 'saeb_mat_ef_1',
    'Nota SAEB - 2019 Língua Portuguesa': 'saeb_port_ef_1',
    'IDEB\n2019\n(N x P)': 'ideb_ef_1'
}).reset_index(drop=True)

dados_ideb_ef_1.head()

In [ ]:
#Importa dados do IDEB (anos finais)
dados_ideb_ef_2 = pd.read_excel('datasets/raw/divulgacao_anos_finais_municipios_2019.xlsx',
                                header = [6, 7, 8])

dados_ideb_ef_2 = remove_multilevel_index(dados_ideb_ef_2)
dados_ideb_ef_2 = dados_ideb_ef_2[dados_ideb_ef_2['Rede'] == 'Pública'][[
    'Código do Município',
    'Taxa de Aprovação - 2019  9º',
    'Nota SAEB - 2019 Matemática',
    'Nota SAEB - 2019 Língua Portuguesa',
    'IDEB\n2019\n(N x P)'
]].rename(columns={
    'Código do Município': 'cod_mun',
    'Taxa de Aprovação - 2019  9º': 'tx_aprov_ef_2',
    'Nota SAEB - 2019 Matemática': 'saeb_mat_ef_2',
    'Nota SAEB - 2019 Língua Portuguesa': 'saeb_port_ef_2',
    'IDEB\n2019\n(N x P)': 'ideb_ef_2'
}).reset_index(drop=True)

dados_ideb_ef_2.head()

In [ ]:
#Importa dados do IDEB (ensino médio)
dados_ideb_em = pd.read_excel('datasets/raw/divulgacao_ensino_medio_municipios_2019.xlsx',
                              header = [6, 7, 8])

dados_ideb_em = remove_multilevel_index(dados_ideb_em)
dados_ideb_em = dados_ideb_em[dados_ideb_em['Rede'] == 'Pública'][[
    'Código do Município',
    'Taxa de Aprovação - 2019  Total',
    'Nota SAEB - 2019 Matemática',
    'Nota SAEB - 2019 Língua Portuguesa',
    'IDEB\n2019\n(N x P)'
]].rename(columns={
    'Código do Município': 'cod_mun',
    'Taxa de Aprovação - 2019  Total': 'tx_aprov_em',
    'Nota SAEB - 2019 Matemática': 'saeb_mat_em',
    'Nota SAEB - 2019 Língua Portuguesa': 'saeb_port_em',
    'IDEB\n2019\n(N x P)': 'ideb_em'
}).reset_index(drop=True)

dados_ideb_em.head()

## 1.3. Indicadores sociais

In [ ]:
#Importa variáveis de controle para os municípios
indicadores_sociais = pd.read_csv("datasets/raw/indicadores_sociais.csv",
                                  sep = ',')

#Vamos selecionar variáveis de anos diferentes:
#Censo 2010 (% de pobres, taxa de analfabetismo)
#DataSUS 2017 (Taxa de mortalidade infantil, % de meninas que tiveram filhos)
#RAIS 2016 (PIB per capita)
indicadores_sociais = indicadores_sociais[[
    'Territorialidades',
    'Esperança de vida ao nascer 2010',
    'Taxa de analfabetismo - 15 anos ou mais de idade 2010',
    '% de pobres 2010', 
    'Produto Interno Bruto per capita 2016',
    'Taxa de mortalidade infantil 2017',
    '% de meninas de 10 a 14 anos de idade que tiveram filhos 2017'
]].rename(columns={
    'Territorialidades': 'nom_mun',
    'Esperança de vida ao nascer 2010': 'exp_vida',
    'Taxa de analfabetismo - 15 anos ou mais de idade 2010': 'analfabetismo',
    '% de pobres 2010': 'tx_pobreza',
    'Produto Interno Bruto per capita 2016': 'pib_per_capita',
    'Taxa de mortalidade infantil 2017': 'mortalidade_inf',
    '% de meninas de 10 a 14 anos de idade que tiveram filhos 2017': 'maternidade_inf'
}).reset_index(drop=True)

indicadores_sociais['sig_uf'] = indicadores_sociais['nom_mun'].apply(lambda x: str(x)[-3:-1])
indicadores_sociais['nom_mun'] = indicadores_sociais['nom_mun'].apply(lambda x: unidecode(str(x)[:-5].upper()))

indicadores_sociais.head()

### Juntando tudo

In [ ]:
df = dados_ideb_ef_1.merge(dados_ideb_ef_2, on='cod_mun', how='inner')\
                    .merge(dados_ideb_em, on='cod_mun', how='inner')\
                    .merge(dicionario_ibge.drop(columns=['cod_ap', 'cod_setor']).drop_duplicates(), on='cod_mun', how='inner')\
                    .merge(indicadores_sociais, on=['nom_mun', 'sig_uf'], how='inner')\
                    .merge(internet_connection_grp, on=['cod_mun', 'nom_mun', 'sig_uf'], how='inner')

df = df.reindex(columns=(
    ['cod_mun', 'nom_mun', 'sig_uf'] + list(
        [x for x in df.columns.tolist() if x not in ['cod_mun', 'nom_mun', 'sig_uf']]
    )
))

df.to_csv('datasets/dataset_pre_limpeza.parquet', index=False)
df.head()

# 2. Limpeza dos dados

## 2.1. Dados de educação

In [ ]:
col_list = [
    'tx_aprov_ef_1',
    'saeb_mat_ef_1',
    'saeb_port_ef_1',
    'ideb_ef_1',
    'tx_aprov_ef_2',
    'saeb_mat_ef_2',
    'saeb_port_ef_2',
    'ideb_ef_2',
    'tx_aprov_em',
    'saeb_mat_em',
    'saeb_port_em',
    'ideb_em'
]

# Padroniza linhas vazias
for col in col_list:
    df[col] = df[col].replace('-', np.nan)
    df[col] = df[col].replace('ND', np.nan)
    df[col] = df[col].astype(float)

## 2.2. Indicadores sociais

In [ ]:
col_list = [
    'exp_vida',
    'analfabetismo',
    'tx_pobreza',
    'pib_per_capita',
    'mortalidade_inf',
    'maternidade_inf'
]

# Padroniza linhas vazias
for col in col_list:
    df[col] = df[col].fillna(np.nan)
    df[col] = df[col].astype(float)

## 2.3. Dados de internet

In [ ]:
col_list = [x for x in df.columns.tolist() if 'tcp_range' in x]

# Padroniza linhas vazias
for col in col_list:
    df[col] = df[col].fillna(0)
    df[col] = df[col].astype(float)

## 2.4. Salva o DF limpo para 

In [ ]:
df.to_parquet('datasets/dataset_pos_limpeza.parquet', index=False)